In [1]:
# Ensures that there is enough memory allocation for the model to load
import os
import torch
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()

In [2]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info

# default: Load the model on the available device(s)
model_QWEN = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-VL-7B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

# default processor
processor_QWEN = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct", use_fast=True)

/home/jovyan/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.53s/it]


In [6]:
import gc
import json
# Path to the directory (library)
directory_path = "/home/jovyan/images"
# List all files in the directory
files = os.listdir(directory_path)

# Filter out directories and show only files
files = [file for file in files if os.path.isfile(os.path.join(directory_path, file))]

data = {}

# Print the list of files
for index, file in enumerate(files):
    
    print(f"Processing image {file}")
    
    image_path = os.path.join(directory_path, file)

    # Prompt for the Qwen
    prompt = f"""
    TASK : Object Character Recognition by Region.
    Detect all text in the image.
    Output:
        - Do not include any header.
        - Include only the text detected and nothing else.
        - Ensure a consistent output.
    """


    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": f"{image_path}",
                    "resized_height": 280,
                    "resized_width": 420, 
                },
                {"type": "text", "text": f"{prompt}"},
            ],
        }
    ]

    # Preparation for inference
    text = processor_QWEN.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor_QWEN(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    # Inference: Generation of the output
    with torch.no_grad():
        generated_ids = model_QWEN.generate(**inputs, max_new_tokens=100)
        generated_ids_trimmed = [
            out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        output_text = processor_QWEN.batch_decode(
            generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )
    # Converts all the text into lower caps.
    data[file] = output_text[0].split()
    
    print(f"Successfully processed Image {file} ,{index+1}/{len(files)} Images Processed")
    del inputs, generated_ids, image_inputs, video_inputs
    gc.collect()  # Force garbage collection

    
with open("QwenOCR.json", "w") as f:
    json.dump(data,f,indent=4)
    
print(f"All {len(files)} have been processed.")

Processing image 588.JPG
Successfully processed Image 588.JPG ,1/265 Images Processed
Processing image 708.png
Successfully processed Image 708.png ,2/265 Images Processed
Processing image 603.jpg
Successfully processed Image 603.jpg ,3/265 Images Processed
Processing image 546.jpg
Successfully processed Image 546.jpg ,4/265 Images Processed
Processing image 728.png
Successfully processed Image 728.png ,5/265 Images Processed
Processing image 721.png
Successfully processed Image 721.png ,6/265 Images Processed
Processing image 759.jpg
Successfully processed Image 759.jpg ,7/265 Images Processed
Processing image 619.jpg
Successfully processed Image 619.jpg ,8/265 Images Processed
Processing image 572.jpg
Successfully processed Image 572.jpg ,9/265 Images Processed
Processing image 614.jpg
Successfully processed Image 614.jpg ,10/265 Images Processed
Processing image 518.jpg
Successfully processed Image 518.jpg ,11/265 Images Processed
Processing image 687.jpg
Successfully processed Imag